# Matching AMM data collected from WoS  with Scopus

# Import packages, keys

In [1]:
import pybliometrics
import csv
import pandas as pd
from pybliometrics.scopus import ScopusSearch
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#pybliometrics.scopus.utils.create_config()

In [11]:
key="your_api_key"
inst="your_institutional_token"

# 1975-1979

In [105]:
data=pd.read_csv("your_data_path", encoding='latin1',sep=";")
data

,X,ID_Article_citant,Date_Article_citant,Auteur_Document_cite,Titre_Document_cite,Volume_Document_cite,Date_Document_cite,nom
1,1,41025929,1975,AARON-H,SHELTER SUBSIDIES WH,NaN,1972.0,NaN
2,2,41822237,1977,AARON-H,WHO PAYS PROPERTY TA,NaN,1975.0,NaN
3,3,41863953,1977,AARON-H,AM ECON REV PAP P,64.0,1974.0,NaN
4,4,43053567,1979,AARON-H,WHO PAYS PROPERTY TA,NaN,1975.0,NaN
5,5,40956854,1975,AARON-HJ,WESTERN ECONOMIC J,9.0,1971.0,Western Journal of Agricultural Economics
...,...,...,...,...,...,...,...,...
3533,3533,42987891,1978,ZELLNER-A,J AM STATISTICAL ASS,57.0,1962.0,Journal of the American Statistical Association
3534,3534,43467487,1979,ZELLNER-A,J AM STATISTICAL ASS,57.0,1962.0,Journal of the American Statistical Association
3535,3535,43487652,1979,ZELLNER-A,ECONOMETRICA,30.0,1962.0,Econometrica
3536,3536,43487652,1979,ZELLNER-A,ECONOMETRICA,33.0,1965.0,Econometrica


In [106]:
data['Auteur_req']=data['Auteur_Document_cite'].str.replace('-',',')

#Colonne avec juste première initiale du nom de l'auteur
data['pren_1ltr']=data['Auteur_req'].str.rsplit(',').str[-1].str[0]

#Colonne avec juste nom de famille
data['nom_aut']=data['Auteur_req'].str.rsplit(',').str[0]

#Colonne des années en string
data['pubyear']=data['Date_Document_cite'].astype(str).str.rsplit('.').str[0]
data['pubyear']=data['pubyear'].replace('nan', np.NaN)

#Volume en string
data['vol']=data['Volume_Document_cite'].astype(str).str.rsplit('.').str[0]
data['vol']=data['vol'].replace('nan', np.NaN)

In [107]:
data['req1']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['nom']+'") AND PUBYEAR IS ' + data['pubyear']
data['req2']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['nom']+'")'
data['req3']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']

In [116]:
resultats=[]

for i in range(1,len(data)+1):
    try:
        if pd.isna(data['nom'][i])==True and pd.isna(data['pubyear'][i])==False: #pas de journal mais année : auteur/auteur+année
            search3=ScopusSearch(data['req3'][i]).results
            resultats.append(search3)

        elif pd.isna(data['nom'][i])==False and pd.isna(data['pubyear'][i])==True:#journal mais pas année : auteur+journal
            search2=ScopusSearch(data['req2'][i]).results
            resultats.append(search2)

        else: #journal et année : auteur+journal/auteur+journal+année
            search1=ScopusSearch(data['req1'][i]).results    
            if search1 is None:
                search2=ScopusSearch(data['req2'][i]).results 
                resultats.append(search2)
            else:
                resultats.append(search1)
    
    except:
        resultats.append(None)

In [117]:
#Combien d'articles sont trouvés ?
nb_art=[]
for i in range(len(resultats)):
    if resultats[i] is None:
        nb_art.append(0)
    else:
        nb_art.append(len(resultats[i]))
        
#print(nb_art)
print('un seul article est trouvé pour',sum(np.array(nb_art)==1),'observations')
print('aucun article trouvé pour', sum(np.array(nb_art)==0),'observations')
print('plus de deux articles trouvé pour',sum(np.array(nb_art)>=2),'observations')

un seul article est trouvé pour 782 observations
aucun article trouvé pour 1763 observations
plus de deux articles trouvé pour 992 observations


In [120]:
data['nb_art']=nb_art

In [118]:
#création des requêtes : auteur + année + volume + journal/ auteur + volume + journal/ auteur + année + discipline + volume
# / auteur + année + discipline 
data['req4']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['nom']+'") AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+")"
data['req5']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['nom']+'") AND VOLUME('+data['vol']+')'
data['req6']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+") AND SUBJAREA(ECON)"
data['req7']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND SUBJAREA(ECON)"

In [ ]:
#on utilise ces requêtes pour les observations sans résultat
resultats2=[]

for i in range(len(data)):
    if data['nb_art'][i+1]==1:
        resultats2.append(resultats[i])
        print("on garde ligne ",i)
    else:
        try:
            if pd.isna(data['nom'][i+1])==False and pd.isna(data['pubyear'][i+1])==False and pd.isna(data['vol'][i+1])==False:
                search4=ScopusSearch(data['req4'][i+1]).results
                print("on est à search4, la longueur testée est",len(search4),"ligne",i)
                if len(search4)==1:
                    resultats2.append(search4)
                else :
                    search6=ScopusSearch(data['req6'][i+1]).results
                    if len(search6)==1:
                        resultats2.append(search6)
                        print("on est passé à search 6 pour ",i)
                    else:
                        resultats2.append(resultats[i])
                
            elif pd.isna(data['nom'][i+1])==False and pd.isna(data['pubyear'][i+1])==True and pd.isna(data['vol'][i+1])==False:
                search5=ScopusSearch(data['req4'][i+1]).results
                if len(search5)==1:
                    resultats2.append(search5)
                    print("SEARCH5",i)
                else:
                    resultats2.append(resultats[i])
                
            elif pd.isna(data['pubyear'][i+1])==False:
                search7=ScopusSearch(data['req7'][i+1]).results
                if len(search7)==1:
                    resultats2.append(search7)
                    print("SEARCH7",i)
                else:
                    resultats2.append(resultats[i])
            
            else:
                resultats2.append(resultats[i])
                print("pas mieux on retourne à ancienne ligne pour",i)
            

        except:
            resultats2.append(resultats[i])

In [122]:
nb_art2=[]
for i in range(len(resultats2)):
    if resultats2[i] is None:
        nb_art2.append(0)
    else:
        nb_art2.append(len(resultats2[i]))


auteurs2=[]
for i in range(len(resultats2)):
    if nb_art2[i]!=1:
        auteurs2.append(None)
    else:
        auteurs2.append(resultats2[i][0].author_names)

print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/len(data)*100,'% des observations')#22%
print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/sum(pd.notna(data['Volume_Document_cite']))*100,'% des articles')

sum(np.array(auteurs)!=None)

sum(np.array(auteurs2)!=None)

on récupère les auteurs de façon certaine pour 25.64320045236076 % des observations
on récupère les auteurs de façon certaine pour 48.711063372717504 % des articles


907

In [123]:
dfauteurs_7579=pd.DataFrame(auteurs2)

dfauteurs_7579.to_csv('dfauteurs_7579.csv')

# 1985-1989

In [124]:
data=pd.read_csv("your_data_path", encoding='latin1',sep=";")
data

,X,ID_Article_citant,Date_Article_citant,Auteur_Document_cite,Titre_Document_cite,Volume_Documant_cite,Date_Document_cite,nom
1,1,5630278,1986,AARON-H,SHELTER SUBSIDIES WH,NaN,1972.0,NaN
2,2,7014194,1988,AARON-H,AM EC REV,60.0,1970.0,American Economic Review
3,3,7984593,1989,AARON-H,ECONOMETRICA,38.0,1970.0,Econometrica
4,4,7984593,1989,AARON-H,ECONOMETRICA,44.0,1976.0,Econometrica
5,5,4469987,1985,AJO-R,ACTA GEOGRAPHICA,18.0,1965.0,Acta Geographica
...,...,...,...,...,...,...,...,...
3363,3363,7755128,1988,ZIELINSKI-K,ENVIRON PLANN A,11.0,1979.0,Environment and Planning A
3364,3364,8443171,1989,ZIELINSKI-K,ENVIRON PLANN A,11.0,1979.0,Environment and Planning A
3365,3365,4330146,1985,ZIPF-GK,HUMAN BEHAVIOR PRINC,NaN,1949.0,NaN
3366,3366,4940699,1985,ZIPF-GK,HUMAN RELATIONS PRIN,NaN,1949.0,NaN


In [126]:
data['Auteur_req']=data['Auteur_Document_cite'].str.replace('-',',')

#Colonne avec juste première initiale du nom de l'auteur
data['pren_1ltr']=data['Auteur_req'].str.rsplit(',').str[-1].str[0]

#Colonne avec juste nom de famille
data['nom_aut']=data['Auteur_req'].str.rsplit(',').str[0]

#Colonne des années en string
data['pubyear']=data['Date_Document_cite'].astype(str).str.rsplit('.').str[0]
data['pubyear']=data['pubyear'].replace('nan', np.NaN)

#Volume en string
data['vol']=data['Volume_Documant_cite'].astype(str).str.rsplit('.').str[0]
data['vol']=data['vol'].replace('nan', np.NaN)

In [127]:
data['req1']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['nom']+'") AND PUBYEAR IS ' + data['pubyear']
data['req2']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['nom']+'")'
data['req3']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']

In [128]:
resultats=[]

for i in range(1,len(data)+1):
    try:
        if pd.isna(data['nom'][i])==True and pd.isna(data['pubyear'][i])==False: #pas de journal mais année : auteur/auteur+année
            search3=ScopusSearch(data['req3'][i]).results
            resultats.append(search3)

        elif pd.isna(data['nom'][i])==False and pd.isna(data['pubyear'][i])==True:#journal mais pas année : auteur+journal
            search2=ScopusSearch(data['req2'][i]).results
            resultats.append(search2)

        else: #journal et année : auteur+journal/auteur+journal+année
            search1=ScopusSearch(data['req1'][i]).results    
            if search1 is None:
                search2=ScopusSearch(data['req2'][i]).results 
                resultats.append(search2)
            else:
                resultats.append(search1)
    
    except:
        resultats.append(None)

In [129]:
#Combien d'articles sont trouvés ?
nb_art=[]
for i in range(len(resultats)):
    if resultats[i] is None:
        nb_art.append(0)
    else:
        nb_art.append(len(resultats[i]))
        
#print(nb_art)
print('un seul article est trouvé pour',sum(np.array(nb_art)==1),'observations')
print('aucun article trouvé pour', sum(np.array(nb_art)==0),'observations')
print('plus de deux articles trouvé pour',sum(np.array(nb_art)>=2),'observations')

un seul article est trouvé pour 931 observations
aucun article trouvé pour 1440 observations
plus de deux articles trouvé pour 996 observations


In [130]:
data['nb_art']=nb_art

In [131]:
#création des requêtes : auteur + année + volume + journal/ auteur + volume + journal/ auteur + année + discipline + volume
# / auteur + année + discipline 
data['req4']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['nom']+'") AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+")"
data['req5']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['nom']+'") AND VOLUME('+data['vol']+')'
data['req6']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+") AND SUBJAREA(ECON)"
data['req7']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND SUBJAREA(ECON)"

In [ ]:
#on utilise ces requêtes pour les observations sans résultat
resultats2=[]

for i in range(len(data)):
    if data['nb_art'][i+1]==1:
        resultats2.append(resultats[i])
        print("on garde ligne ",i)
    else:
        try:
            if pd.isna(data['nom'][i+1])==False and pd.isna(data['pubyear'][i+1])==False and pd.isna(data['vol'][i+1])==False:
                search4=ScopusSearch(data['req4'][i+1]).results
                print("on est à search4, la longueur testée est",len(search4),"ligne",i)
                if len(search4)==1:
                    resultats2.append(search4)
                else :
                    search6=ScopusSearch(data['req6'][i+1]).results
                    if len(search6)==1:
                        resultats2.append(search6)
                        print("on est passé à search 6 pour ",i)
                    else:
                        resultats2.append(resultats[i])
                
            elif pd.isna(data['nom'][i+1])==False and pd.isna(data['pubyear'][i+1])==True and pd.isna(data['vol'][i+1])==False:
                search5=ScopusSearch(data['req4'][i+1]).results
                if len(search5)==1:
                    resultats2.append(search5)
                    print("SEARCH5",i)
                else:
                    resultats2.append(resultats[i])
                
            elif pd.isna(data['pubyear'][i+1])==False:
                search7=ScopusSearch(data['req7'][i+1]).results
                if len(search7)==1:
                    resultats2.append(search7)
                    print("SEARCH7",i)
                else:
                    resultats2.append(resultats[i])
            
            else:
                resultats2.append(resultats[i])
                print("pas mieux on retourne à ancienne ligne pour",i)
            

        except:
            resultats2.append(resultats[i])

In [134]:
nb_art2=[]
for i in range(len(resultats2)):
    if resultats2[i] is None:
        nb_art2.append(0)
    else:
        nb_art2.append(len(resultats2[i]))


auteurs2=[]
for i in range(len(resultats2)):
    if nb_art2[i]!=1:
        auteurs2.append(None)
    else:
        auteurs2.append(resultats2[i][0].author_names)

print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/len(data)*100,'% des observations')#22%
print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/sum(pd.notna(data['Volume_Documant_cite']))*100,'% des articles')

sum(np.array(auteurs)!=None)

sum(np.array(auteurs2)!=None)

on récupère les auteurs de façon certaine pour 34.80843480843481 % des observations
on récupère les auteurs de façon certaine pour 56.89320388349515 % des articles


1172

In [135]:
dfauteurs_8589=pd.DataFrame(auteurs2)

dfauteurs_8589.to_csv('dfauteurs_8589.csv')

# 2000-2004

In [4]:
data=pd.read_csv("your_data_path", encoding='latin1',sep=";")
data

,X,Citing_ID,Citing_Date,Cited_Author,Cited_Document,Cited_Vol,Cited_Yr,Cited_Doc_True
0,1,19728188,2000,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science
1,2,19837085,2000,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science
2,3,20956590,2001,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science
3,4,21616584,2002,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science
4,5,22668585,2003,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science
...,...,...,...,...,...,...,...,...
6584,6585,23264917,2003,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN
6585,6586,23908457,2004,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN
6586,6587,23908461,2004,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN
6587,6588,21585118,2002,ZUBRINSKY-CL,SOC SCI RES,25.0,1996.0,Social Science Research


In [5]:
data['Auteur_req']=data['Cited_Author'].str.replace('-',',')

#Colonne avec juste première initiale du nom de l'auteur
data['pren_1ltr']=data['Auteur_req'].str.rsplit(',').str[-1].str[0]

#Colonne avec juste nom de famille
data['nom_aut']=data['Auteur_req'].str.rsplit(',').str[0]

#Colonne des années en string
data['pubyear']=data['Cited_Yr'].astype(str).str.rsplit('.').str[0]
data['pubyear']=data['pubyear'].replace('nan', np.NaN)

#Volume en string
data['vol']=data['Cited_Vol'].astype(str).str.rsplit('.').str[0]
data['vol']=data['vol'].replace('nan', np.NaN)

In [6]:
data

,X,Citing_ID,Citing_Date,Cited_Author,Cited_Document,Cited_Vol,Cited_Yr,Cited_Doc_True,Auteur_req,pren_1ltr,nom_aut,pubyear,vol
0,1,19728188,2000,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science,"ABDELRAHMAN,HM",H,ABDELRAHMAN,1990,30
1,2,19837085,2000,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science,"ABDELRAHMAN,HM",H,ABDELRAHMAN,1990,30
2,3,20956590,2001,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science,"ABDELRAHMAN,HM",H,ABDELRAHMAN,1990,30
3,4,21616584,2002,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science,"ABDELRAHMAN,HM",H,ABDELRAHMAN,1990,30
4,5,22668585,2003,ABDELRAHMAN-HM,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science,"ABDELRAHMAN,HM",H,ABDELRAHMAN,1990,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,6585,23264917,2003,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN,"ZIPF,GK",G,ZIPF,1949,NaN
6585,6586,23908457,2004,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN,"ZIPF,GK",G,ZIPF,1949,NaN
6586,6587,23908461,2004,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN,"ZIPF,GK",G,ZIPF,1949,NaN
6587,6588,21585118,2002,ZUBRINSKY-CL,SOC SCI RES,25.0,1996.0,Social Science Research,"ZUBRINSKY,CL",C,ZUBRINSKY,1996,25


In [7]:
#Formulation de la requête 
#exemple : AUTHOR-NAME ( anderson,r )  AND  SRCTITLE ( "URBAN STUDIES" )  AND  PUBYEAR  =  1971 

data['req1']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['Cited_Doc_True']+'") AND PUBYEAR IS ' + data['pubyear']
data['req2']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['Cited_Doc_True']+'")'
data['req3']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']

In [8]:
#Exemple pour la première ligne
data['req1'][1]

'AUTHOR-NAME(ABDELRAHMAN,H) AND  SRCTITLE("Journal of Regional Science") AND PUBYEAR IS 1990'

In [9]:
resultats=[]

for i in range(1,len(data)+1):
    try:
        if pd.isna(data['Cited_Doc_True'][i])==True and pd.isna(data['pubyear'][i])==False: #pas de journal mais année : auteur/auteur+année
            search3=ScopusSearch(data['req3'][i]).results
            resultats.append(search3)

        elif pd.isna(data['Cited_Doc_True'][i])==False and pd.isna(data['pubyear'][i])==True:#journal mais pas année : auteur+journal
            search2=ScopusSearch(data['req2'][i]).results
            resultats.append(search2)

        else: #journal et année : auteur+journal/auteur+journal+année
            search1=ScopusSearch(data['req1'][i]).results    
            if search1 is None:
                search2=ScopusSearch(data['req2'][i]).results 
                resultats.append(search2)
            else:
                resultats.append(search1)
    
    except:
        resultats.append(None)

In [10]:
#Combien d'articles sont trouvés ?
nb_art=[]
for i in range(len(resultats)):
    if resultats[i] is None:
        nb_art.append(0)
    else:
        nb_art.append(len(resultats[i]))
        
#print(nb_art)
print('un seul article est trouvé pour',sum(np.array(nb_art)==1),'observations')
print('aucun article trouvé pour', sum(np.array(nb_art)==0),'observations')
print('plus de deux articles trouvé pour',sum(np.array(nb_art)>=2),'observations')

un seul article est trouvé pour 3088 observations
aucun article trouvé pour 1196 observations
plus de deux articles trouvé pour 2305 observations


In [12]:
data['nb_art']=nb_art

In [14]:
#création des requêtes : auteur + année + volume + journal/ auteur + volume + journal/ auteur + année + discipline + volume
# / auteur + année + discipline 
data['req4']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['Cited_Doc_True']+'") AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+")"
data['req5']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['Cited_Doc_True']+'") AND VOLUME('+data['vol']+')'
data['req6']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+") AND SUBJAREA(ECON)"
data['req7']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND SUBJAREA(ECON)"

In [ ]:
#on utilise ces requêtes pour les observations sans résultat
resultats2=[]

for i in range(len(data)):
    if data['nb_art'][i+1]==1:
        resultats2.append(resultats[i])
        print("on garde ligne ",i)
    else:
        try:
            if pd.isna(data['Cited_Doc_True'][i+1])==False and pd.isna(data['pubyear'][i+1])==False and pd.isna(data['vol'][i+1])==False:
                search4=ScopusSearch(data['req4'][i+1]).results
                print("on est à search4, la longueur testée est",len(search4),"ligne",i)
                if len(search4)==1:
                    resultats2.append(search4)
                else :
                    search6=ScopusSearch(data['req6'][i+1]).results
                    if len(search6)==1:
                        resultats2.append(search6)
                        print("on est passé à search 6 pour ",i)
                    else:
                        resultats2.append(resultats[i])
                
            elif pd.isna(data['Cited_Doc_True'][i+1])==False and pd.isna(data['pubyear'][i+1])==True and pd.isna(data['vol'][i+1])==False:
                search5=ScopusSearch(data['req4'][i+1]).results
                if len(search5)==1:
                    resultats2.append(search5)
                    print("SEARCH5",i)
                else:
                    resultats2.append(resultats[i])
                
            elif pd.isna(data['pubyear'][i+1])==False:
                search7=ScopusSearch(data['req7'][i+1]).results
                if len(search7)==1:
                    resultats2.append(search7)
                    print("SEARCH7",i)
                else:
                    resultats2.append(resultats[i])
            
            else:
                resultats2.append(resultats[i])
                print("pas mieux on retourne à ancienne ligne pour",i)
            

        except:
            resultats2.append(resultats[i])

In [16]:
nb_art2=[]
for i in range(len(resultats2)):
    if resultats2[i] is None:
        nb_art2.append(0)
    else:
        nb_art2.append(len(resultats2[i]))


auteurs2=[]
for i in range(len(resultats2)):
    if nb_art2[i]!=1:
        auteurs2.append(None)
    else:
        auteurs2.append(resultats2[i][0].author_names)

print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/len(data)*100,'% des observations')#22%
print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/sum(pd.notna(data['Cited_Vol']))*100,'% des articles')

on récupère les auteurs de façon certaine pour 51.26726362118683 % des observations
on récupère les auteurs de façon certaine pour 79.70740915526193 % des articles


In [19]:
sum(np.array(auteurs2)!=None)

3378

In [20]:
dfauteurs_0004=pd.DataFrame(auteurs2)

dfauteurs_0004.to_csv('dfauteurs_0004.csv')

# 2005-2009

In [136]:
data=pd.read_csv("your_data_path", encoding='latin1',sep=";")
data

,X,ID_Article_citant,Date_Article_citant,Auteur_Document_cite,Titre_Document_cite,Volume_Document_cite,Date_Document_cite,nom
1,1,31633146,2009,AARONSON-D,EC PERSPECTIVES,25.0,2001.0,Review of Economic Perspectives
2,2,32652309,2009,AARONSON-D,J URBAN ECON,49.0,2001.0,Journal of Urban Economics
3,3,32737930,2009,AARONSON-D,J URBAN ECON,49.0,2001.0,Journal of Urban Economics
4,4,30333917,2008,ABDELRAHMAN-H,ADV URBAN EC,NaN,2004.0,NaN
5,5,28920921,2007,ABDELRAHMAN-H,J REGIONAL SCI,30.0,1990.0,Journal of Regional Science
...,...,...,...,...,...,...,...,...
6112,6112,25471138,2005,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN
6113,6113,28607848,2007,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN
6114,6114,32318757,2009,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN
6115,6115,43807417,2009,ZIPF-GK,HUMAN BEHAV PRINCIPL,NaN,1949.0,NaN


In [137]:
data['Auteur_req']=data['Auteur_Document_cite'].str.replace('-',',')

#Colonne avec juste première initiale du nom de l'auteur
data['pren_1ltr']=data['Auteur_req'].str.rsplit(',').str[-1].str[0]

#Colonne avec juste nom de famille
data['nom_aut']=data['Auteur_req'].str.rsplit(',').str[0]

#Colonne des années en string
data['pubyear']=data['Date_Document_cite'].astype(str).str.rsplit('.').str[0]
data['pubyear']=data['pubyear'].replace('nan', np.NaN)

#Volume en string
data['vol']=data['Volume_Document_cite'].astype(str).str.rsplit('.').str[0]
data['vol']=data['vol'].replace('nan', np.NaN)

In [138]:
data['req1']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['nom']+'") AND PUBYEAR IS ' + data['pubyear']
data['req2']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND  SRCTITLE("'+data['nom']+'")'
data['req3']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']

In [139]:
resultats=[]

for i in range(1,len(data)+1):
    try:
        if pd.isna(data['nom'][i])==True and pd.isna(data['pubyear'][i])==False: #pas de journal mais année : auteur/auteur+année
            search3=ScopusSearch(data['req3'][i]).results
            resultats.append(search3)

        elif pd.isna(data['nom'][i])==False and pd.isna(data['pubyear'][i])==True:#journal mais pas année : auteur+journal
            search2=ScopusSearch(data['req2'][i]).results
            resultats.append(search2)

        else: #journal et année : auteur+journal/auteur+journal+année
            search1=ScopusSearch(data['req1'][i]).results    
            if search1 is None:
                search2=ScopusSearch(data['req2'][i]).results 
                resultats.append(search2)
            else:
                resultats.append(search1)
    
    except:
        resultats.append(None)

In [140]:
#Combien d'articles sont trouvés ?
nb_art=[]
for i in range(len(resultats)):
    if resultats[i] is None:
        nb_art.append(0)
    else:
        nb_art.append(len(resultats[i]))
        
#print(nb_art)
print('un seul article est trouvé pour',sum(np.array(nb_art)==1),'observations')
print('aucun article trouvé pour', sum(np.array(nb_art)==0),'observations')
print('plus de deux articles trouvé pour',sum(np.array(nb_art)>=2),'observations')

un seul article est trouvé pour 2784 observations
aucun article trouvé pour 1095 observations
plus de deux articles trouvé pour 2237 observations


In [141]:
data['nb_art']=nb_art

In [142]:
#création des requêtes : auteur + année + volume + journal/ auteur + volume + journal/ auteur + année + discipline + volume
# / auteur + année + discipline 
data['req4']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['nom']+'") AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+")"
data['req5']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND SRCTITLE("'+data['nom']+'") AND VOLUME('+data['vol']+')'
data['req6']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND VOLUME("+data['vol']+") AND SUBJAREA(ECON)"
data['req7']="AUTHOR-NAME("+data['nom_aut']+","+data['pren_1ltr']+') AND PUBYEAR IS ' + data['pubyear']+" AND SUBJAREA(ECON)"

In [ ]:
#on utilise ces requêtes pour les observations sans résultat
resultats2=[]

for i in range(len(data)):
    if data['nb_art'][i+1]==1:
        resultats2.append(resultats[i])
        print("on garde ligne ",i)
    else:
        try:
            if pd.isna(data['nom'][i+1])==False and pd.isna(data['pubyear'][i+1])==False and pd.isna(data['vol'][i+1])==False:
                search4=ScopusSearch(data['req4'][i+1]).results
                print("on est à search4, la longueur testée est",len(search4),"ligne",i)
                if len(search4)==1:
                    resultats2.append(search4)
                else :
                    search6=ScopusSearch(data['req6'][i+1]).results
                    if len(search6)==1:
                        resultats2.append(search6)
                        print("on est passé à search 6 pour ",i)
                    else:
                        resultats2.append(resultats[i])
                
            elif pd.isna(data['nom'][i+1])==False and pd.isna(data['pubyear'][i+1])==True and pd.isna(data['vol'][i+1])==False:
                search5=ScopusSearch(data['req4'][i+1]).results
                if len(search5)==1:
                    resultats2.append(search5)
                    print("SEARCH5",i)
                else:
                    resultats2.append(resultats[i])
                
            elif pd.isna(data['pubyear'][i+1])==False:
                search7=ScopusSearch(data['req7'][i+1]).results
                if len(search7)==1:
                    resultats2.append(search7)
                    print("SEARCH7",i)
                else:
                    resultats2.append(resultats[i])
            
            else:
                resultats2.append(resultats[i])
                print("pas mieux on retourne à ancienne ligne pour",i)
            

        except:
            resultats2.append(resultats[i])

In [144]:
nb_art2=[]
for i in range(len(resultats2)):
    if resultats2[i] is None:
        nb_art2.append(0)
    else:
        nb_art2.append(len(resultats2[i]))


auteurs2=[]
for i in range(len(resultats2)):
    if nb_art2[i]!=1:
        auteurs2.append(None)
    else:
        auteurs2.append(resultats2[i][0].author_names)

print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/len(data)*100,'% des observations')#22%
print('on récupère les auteurs de façon certaine pour', sum(np.array(auteurs2)!=None)/sum(pd.notna(data['Volume_Document_cite']))*100,'% des articles')

sum(np.array(auteurs)!=None)

sum(np.array(auteurs2)!=None)

on récupère les auteurs de façon certaine pour 52.223675604970566 % des observations
on récupère les auteurs de façon certaine pour 74.6087362765709 % des articles


3194

In [ ]:
auteurs2

In [145]:
dfauteurs_0509=pd.DataFrame(auteurs2)

dfauteurs_0509.to_csv('dfauteurs_0509.csv')